# Logic Graph Examples

Create interactive graphs of logical formulas with inference edges.

**Smart Sampling:** Use the `max_nodes` parameter to intelligently limit graph size.
The sampler prioritizes: axioms → entailed formulas → tautologies → simpler formulas.

For higher depths (3-4), try `max_nodes=200` for fast, focused visualizations!

In [ ]:
# Example 1: Modus Ponens - x, (x → y) ⊢ y
from boolean_functions import Var, Implies, build_logic_graph, export_to_html

x = Var("x")
y = Var("y")
axioms = [x, Implies(x, y)]

# Build graph with smart sampling (limit to 100 most important nodes)
g = build_logic_graph(["x", "y"], axioms, max_depth=2, include_all=False, max_nodes=100)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Check if y is entailed (it should be via modus ponens)
if str(y) in g.nodes:
    print(f"✓ y is entailed: {g.nodes[str(y)]['entailed']}")

# Export to interactive HTML (automatically saved in graphs/ folder)
export_to_html(g, "modus_ponens.html")

Graph has 198 nodes and 446 edges
✓ y is entailed: True
Interactive graph saved to notebook_modus_ponens.html


In [ ]:
# Example 2: Multiple inference rules in one graph
from boolean_functions import Var, Implies, build_logic_graph, export_to_html

p = Var("p")
q = Var("q")
r = Var("r")

# Axioms: p, (p → q), (q → r)
# This will show MP, HS, and other inference edges
axioms = [p, Implies(p, q), Implies(q, r)]

# Try higher depth with sampling! max_nodes keeps it manageable
g = build_logic_graph(["p", "q", "r"], axioms, max_depth=2, include_all=False, max_nodes=150)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Check derived formulas
for formula_str in ["q", "r", "(p → r)"]:
    if formula_str in g.nodes:
        entailed = g.nodes[formula_str]['entailed']
        print(f"✓ {formula_str} entailed: {entailed}")

export_to_html(g, "complex.html")
print("\n→ Open graphs/complex.html - edges are color-coded by inference rule!")

In [ ]:
# Example 3: Explore formulas with specific axioms
from boolean_functions import Var, And, build_logic_graph, export_to_html

a = Var("a")
b = Var("b")

# Start with a conjunction axiom
axioms = [And(a, b)]

# Try depth 3 with smart sampling!
g = build_logic_graph(["a", "b"], axioms=axioms, max_depth=3, include_all=False, max_nodes=200)

print(f"Total formulas: {g.number_of_nodes()}")

# Count by category
axiom_count = sum(1 for _, d in g.nodes(data=True) if d.get('is_axiom'))
entailed_count = sum(1 for _, d in g.nodes(data=True) if d.get('entailed'))
tautologies = sum(1 for _, d in g.nodes(data=True) if d.get('tautology'))
print(f"Axioms: {axiom_count}")
print(f"Entailed formulas: {entailed_count}")
print(f"Tautologies: {tautologies}")

print("\n→ Look for conjunction elimination (∧E) edges in green!")

export_to_html(g, "conjunction.html")

## Understanding the Graph

**Node colors:**
- 🟠 Orange: Axiom formulas (starting points)
- 🟢 Green: Tautologies (always true)
- 🔵 Blue: Entailed from axioms
- ⚪ Gray: Not entailed

**Edge colors (inference rules):**
- Black: Entailment from axioms
- Red: Modus Ponens (MP)
- Pink: Modus Tollens (MT)
- Purple: Disjunctive Syllogism (DS)
- Indigo: Hypothetical Syllogism (HS)
- Green: Conjunction Elimination (∧E)
- Blue: Disjunction Introduction (∨I)

**Interactive features:**
- Drag nodes to rearrange
- Hover over nodes/edges for details
- Scroll to zoom
- Use navigation buttons in bottom-right


**Smart Sampling:**- Perfect for exploring depth 3-4 without overwhelming the browser!

- Set `max_nodes` to limit graph size- Prioritizes: axioms → entailed → tautologies → simpler formulas